## Parametry

In [10]:
INPUT_FILE = "data_float16_series_24.npy"
LEARNING_RATE = 0.0006
EPOCHS = 5
BATCH_SIZE = 32
SERIES_LENGTH = 24

## Wczytanie danych z pliku

In [11]:
import numpy as np

data = np.load("../resources/" + INPUT_FILE)
print(data)
print(data.shape)

[[[-0.2517 -0.2258  0.9707 ...  0.      0.      0.    ]
  [-0.2517 -0.2478  1.058  ...  0.      0.      0.    ]
  [-0.2517 -0.2698  0.9707 ...  0.      0.      0.    ]
  ...
  [-0.2517 -0.687  -0.1512 ...  0.      0.      0.    ]
  [-0.2517 -0.709   0.5396 ...  0.      0.      0.    ]
  [-0.2517 -0.731   0.5396 ...  0.      0.      0.    ]]

 [[-0.2517 -0.2478  1.058  ...  0.      0.      0.    ]
  [-0.2517 -0.2698  0.9707 ...  0.      0.      0.    ]
  [-0.2517 -0.2478  1.058  ...  0.      0.      0.    ]
  ...
  [-0.2517 -0.709   0.5396 ...  0.      0.      0.    ]
  [-0.2517 -0.731   0.5396 ...  0.      0.      0.    ]
  [-0.2517 -0.731   0.3667 ...  0.      0.      0.    ]]

 [[-0.2517 -0.2698  0.9707 ...  0.      0.      0.    ]
  [-0.2517 -0.2478  1.058  ...  0.      0.      0.    ]
  [-0.2517 -0.2478  0.9707 ...  0.      0.      0.    ]
  ...
  [-0.2517 -0.731   0.5396 ...  0.      0.      0.    ]
  [-0.2517 -0.731   0.3667 ...  0.      0.      0.    ]
  [-0.2517 -0.555   1.058 

## Wydzielenie zbiorów

In [12]:
from sklearn.model_selection import train_test_split

X = np.swapaxes(data[:, :SERIES_LENGTH - 1, :],1,2)
Y = np.swapaxes(data[:, SERIES_LENGTH - 1:, :],1,2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)
print("Dane treningowe: "+ str(X_train.shape))

Dane treningowe: (1294194, 35, 23)


## Definicja sieci

### Własna funkcja straty

In [13]:
def loss_func(y_actual, y_predicted):
    return abs((y_actual[0,0] - y_predicted[0,0]) + \
           abs(y_actual[0,1] - y_predicted[0,1]) + \
           abs(y_actual[0,2] - y_predicted[0,2]) + \
           abs(y_actual[0,3] - y_predicted[0,3]) + \
           abs(y_actual[0,4] - y_predicted[0,4]) + \
           abs(y_actual[0,5] - y_predicted[0,5]) + \
           abs(y_actual[0,6] - y_predicted[0,6]) + \
           abs(y_actual[0,25] - y_predicted[0,25]) + \
           abs(y_actual[0,26] - y_predicted[0,26]) + \
           abs(y_actual[0,27] - y_predicted[0,27]) + \
           abs(y_actual[0,28] - y_predicted[0,28]) + \
           abs(y_actual[0,29] - y_predicted[0,29]) + \
           abs(y_actual[0,30] - y_predicted[0,30]) + \
           abs(y_actual[0,31] - y_predicted[0,31]) + \
           abs(y_actual[0,32] - y_predicted[0,32]) + \
           abs(y_actual[0,33] - y_predicted[0,33]) + \
           abs(y_actual[0,34] - y_predicted[0,34]))/17

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow_core.python.keras.optimizer_v2.rmsprop import RMSProp

model = Sequential()
model.add(Dense(128, input_shape=(35, SERIES_LENGTH - 1), activation="relu"))
model.add(LSTM(units=128, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(35))
model.compile(loss=loss_func, optimizer=RMSProp(lr=LEARNING_RATE), metrics=['accuracy'])

## Uczenie

In [ ]:
model.fit(X_train,Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)

In [ ]:
model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print(np.round(np.squeeze(model.predict(X_test[0:1,:,:])),2))
print("\n")
print(np.round(np.squeeze(Y_test[0:1]),2))

In [ ]:
from keras import losses


#print(losses.mean_squared_error(model.predict(X_test[0:1,:,:], Y_test[0:1])))

print(np.squeeze(model.predict(X_test[0:1,:,:])).shape)
print(np.squeeze(Y_test[0:1]).shape)
print(losses.mean_squared_error(np.squeeze(Y_test[0:1]),np.squeeze(model.predict(X_test[0:1,:,:]))))

## Wykres zmiennosci temperatur

### Prawdziwe temperatury

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.squeeze(X_test[1:2:,1:2,:]))
plt.ylabel('some numbers')
plt.show()

### Przewidziane temperatury

In [ ]:
temps = []
temp_row = np.array(X_test[0:1,:,:]) 
for i in range(24):
    predicted = model.predict(temp_row)
    
    predicted[0,7] = int(i==0 or i==1)
    predicted[0,8] = int(i==2 or i==3)
    predicted[0,9] = int(i==4 or i==5)
    predicted[0,10] = int(i==6 or i==7)
    predicted[0,11] = int(i==8 or i==9)
    predicted[0,12] = int(i==10 or i==11)
    predicted[0,13] = int(i==12 or i==13)
    predicted[0,14] = int(i==14 or i==15)
    predicted[0,15] = int(i==16 or i==17)
    predicted[0,16] = int(i==18 or i==19)
    predicted[0,17] = int(i==20 or i==21)
    predicted[0,18] = int(i==22 or i==23)
    
    predicted[0,19] = temp_row[0, 19, 0]
    predicted[0,20] = temp_row[0, 20, 0]
    predicted[0,21] = temp_row[0, 21, 0]
    predicted[0,22] = temp_row[0, 22, 0]
    predicted[0,23] = temp_row[0, 23, 0]
    predicted[0,24] = temp_row[0, 24, 0]
    
    predicted = np.expand_dims(predicted, axis=2)
    temp_row = np.append(arr=temp_row, values=predicted, axis=2)
    temp_row = temp_row[:,:,1:]
print(temp_row.shape)


plt.plot(np.squeeze(temp_row[:,1:2,:]))
plt.ylabel('some numbers')
plt.show()

## 